In [15]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import sys
import re
import uuid

In [2]:
def _clean_dataframe_strings(df, columns):
    """Helper function to clean string columns."""
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.lower().str.strip()
    return df

In [3]:
df_vgsales = pd.read_csv("vgsales.csv")
df_games = pd.read_csv("games.csv")

In [5]:
# --- Preprocessing and Cleaning for vgsales.csv ---
df_vgsales.dropna(subset=['Year', 'Publisher', 'Name', 'Platform', 'Genre'], inplace=True)
df_vgsales.drop_duplicates(inplace=True)
df_vgsales['Year'] = pd.to_numeric(df_vgsales['Year'], errors='coerce').astype('Int64')
sales_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
for col in sales_cols:
    df_vgsales[col] = pd.to_numeric(df_vgsales[col], errors='coerce')
df_vgsales.dropna(subset=sales_cols, inplace=True)

# Use helper to clean string columns
df_vgsales = _clean_dataframe_strings(df_vgsales, ['Name', 'Platform', 'Genre', 'Publisher'])

In [6]:
# --- Preprocessing and Cleaning for games.csv ---
df_games['Rating'] = df_games['Rating'].apply(
    lambda x: float(re.sub(r'[^0-9.]', '', str(x))) if re.sub(r'[^0-9.]', '', str(x)) else None
)
df_games.dropna(subset=['Rating', 'Release Date', 'Number of Reviews'], inplace=True)
df_games.drop_duplicates(inplace=True)
df_games['Number of Reviews'] = pd.to_numeric(
    df_games['Number of Reviews'].apply(lambda x: re.sub(r'[^0-9]', '', str(x))),
    errors='coerce'
).astype('Int64')
df_games.dropna(subset=['Number of Reviews'], inplace=True)

# Use helper to clean string columns
df_games = _clean_dataframe_strings(df_games, ['Title'])

In [8]:
# --- Merging the two DataFrames ---
df_vgsales.rename(columns={'Name': 'game_title'}, inplace=True)
df_games.rename(columns={'Title': 'game_title'}, inplace=True)

df_merged = pd.merge(df_vgsales, df_games, on='game_title', how='inner', suffixes=('_vgsales', '_games'))

print("Data cleaning and merging complete!")
df_merged

Data cleaning and merging complete!


,Rank,game_title,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,...,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,1,wii sports,wii,2006,sports,nintendo,41.49,29.02,3.77,8.46,...,3.7,775,775,"['Simulator', 'Sport']",Bowl like a pro or go for that home run...\n ...,"[""The perfect arcade game. Another type of gam...",18K,39,320,93
1,1,wii sports,wii,2006,sports,nintendo,41.49,29.02,3.77,8.46,...,3.7,775,775,"['Simulator', 'Sport']",Bowl like a pro or go for that home run...\n ...,"[""The perfect arcade game. Another type of gam...",18K,39,320,93
2,1,wii sports,wii,2006,sports,nintendo,41.49,29.02,3.77,8.46,...,3.7,775,775,"['Simulator', 'Sport']",Bowl like a pro or go for that home run...\n ...,"[""The perfect arcade game. Another type of gam...",18K,39,320,93
3,2,super mario bros.,nes,1985,platform,nintendo,29.08,3.58,6.81,0.77,...,3.5,1.5K,15,"['Adventure', 'Platform']",A side scrolling 2D platformer and first entry...,"[""I actually had no idea this game was so long...",18K,59,733,237
4,3,mario kart wii,wii,2008,racing,nintendo,15.85,12.88,3.79,3.31,...,3.9,1K,1,['Racing'],Mario Kart Wii is a multiplayer-oriented racin...,"[""Completed to fulfill a childhood dream and i...",19K,88,461,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364,16166,deus ex,pc,2000,role-playing,eidos interactive,0.00,0.01,0.00,0.00,...,4.4,899,899,"['RPG', 'Shooter']",A philosophical first-person WRPG set in a dys...,"['i think it was too long sometimes, but a wor...",3.8K,189,2.7K,958
1365,16275,chaos;head noah,psv,2014,adventure,5pb,0.00,0.00,0.01,0.00,...,4.0,84,84,"['Adventure', 'Simulator', 'Visual Novel']",Adventure game of psycho suspense touch that m...,"[""some routes are not very good but the main s...",199,73,215,174
1366,16275,chaos;head noah,psv,2014,adventure,5pb,0.00,0.00,0.01,0.00,...,4.0,84,84,"['Adventure', 'Simulator', 'Visual Novel']",Adventure game of psycho suspense touch that m...,"[""some routes are not very good but the main s...",199,73,215,174
1367,16432,goat simulator,xone,2016,simulation,koch media,0.00,0.01,0.00,0.00,...,2.4,337,337,"['Adventure', 'Indie', 'Simulator']",Goat Simulator is a third-person perspective g...,"['XD', 'não tenho o menor interesse em jogar',...",8.4K,22,492,125


In [20]:
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'video_games_db'
}

# Create a connection engine using SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

# Use the engine to connect and populate the table
with engine.connect() as conn:
    print("Creating and populating the 'video_games' table...")
    # to_sql simplifies the entire process. if_exists='replace' drops and recreates the table.
    df_merged.to_sql('video_games_db', conn, if_exists='replace', index=False)
    print("Database setup and population successful!")

Creating and populating the 'video_games' table...
Database setup and population successful!
